# Label Functions

In [1]:
%load_ext autoreload
%autoreload 2
from cord import ResearchPapers
from cord.core import describe_dataframe, find_data_dir, SARS_COV_2_DATE
from pathlib import Path, PurePath
import pandas as pd
pd.options.display.max_colwidth = 2000
pd.options.display.max_rows = 200

In [2]:
research_papers = ResearchPapers.load()
metadata = research_papers.metadata
#metadata = pd.read_csv(PurePath(find_data_dir()) / 'metadata.csv')
#describe_dataframe(metadata)

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 40.0 seconds


In [3]:
metadata = research_papers.metadata.drop(columns=['index_tokens'])

In [4]:
covid_papers = research_papers.covid_related()

In [5]:
covid_papers

,title,abstract,journal,authors,published,when
92,A mathematical model for simulating the transmission of Wuhan novel Coronavirus,"As reported by the World Health Organization, a novel coronavirus (2019-nCoV) was identified as the causative virus of Wuhan pneumonia of unknown etiology by Chinese authorities on 7 January, 2020. In this study, we developed a Bats-Hosts-Reservoir-People transmission network model for simulating the potential transmission from the infection source (probable be bats) to the human infection. Since the Bats-Hosts-Reservoir network was hard to explore clearly and public concerns were focusing on the transmission from a seafood market (reservoir) to people, we simplified the model as Reservoir-People transmission network model. The basic reproduction number (R0) was calculated from the RP model to assess the transmissibility of the 2019-nCoV.",,Tianmu Chen; Jia Rui; Qiupeng Wang; Zeyu Zhao; Jing-An Cui; Ling Yin,2020-01-19,2 months ago
93,"Genomic and protein structure modelling analysis depicts the origin and infectivity of 2019-nCoV, a new coronavirus which caused a pneumonia outbreak in Wuhan, China","Detailed genomic and structure-based analysis of a new coronavirus, namely 2019-nCoV, showed that the new virus is a new type of bat coronavirus and is genetically fairly distant from the human SARS coronavirus. Structure analysis of the spike (S) protein of this new virus showed that its S protein only binds weakly to the ACE2 receptor on human cells whereas the human SARS coronavirus exhibits strongly affinity to the ACE receptor. These findings suggest that the new virus does not readily transmit between humans and should theoretically not able to cause very serious human infection. These data are important to guide design of infection control policy and inform the public on the nature of threat imposed by 2019-nCov when results of direct laboratory tests on this virus are not expected to be available in the near future.",,Ning Dong; Xuemei Yang; Lianwei Ye; Kaichao Chen; Edward Wai-Chi Chan; Mengsu Yang; Sheng Chen,2020-01-21,2 months ago
94,Host and infectivity prediction of Wuhan 2019 novel coronavirus using deep learning algorithm,"The recent outbreak of pneumonia in Wuhan, China caused by the 2019 Novel Coronavirus (2019-nCoV) emphasizes the importance of detecting novel viruses and predicting their risks of infecting people. In this report, we introduced the VHP (Virus Host Prediction) to predict the potential hosts of viruses using deep learning algorithm. Our prediction suggests that 2019-nCoV has close infectivity with other human coronaviruses, especially the severe acute respiratory syndrome coronavirus (SARS-CoV), Bat SARS-like Coronaviruses and the Middle East respiratory syndrome coronavirus (MERS-CoV). Based on our prediction, compared to the Coronaviruses infecting other vertebrates, bat coronaviruses are assigned with more similar infectivity patterns with 2019-nCoVs. Furthermore, by comparing the infectivity patterns of all viruses hosted on vertebrates, we found mink viruses show a closer infectivity pattern to 2019-nCov. These consequences of infectivity pattern analysis illustrate that bat and mink may be two candidate reservoirs of 2019-nCov.These results warn us to beware of 2019-nCoV and guide us to further explore the properties and reservoir of it.One Sentence It is of great value to identify whether a newly discovered virus has the risk of infecting human. Guo et al. proposed a virus host prediction method based on deep learning to detect what kind of host a virus can infect with DNA sequence as input. Applied to the Wuhan 2019 Novel Coronavirus, our prediction demonstrated that several vertebrate-infectious coronaviruses have strong potential to infect human. This method will be helpful in future viral analysis and early prevention and control of viral pathogens.",,Qian Guo; Mo Li; Chunhui Wang; Peihong Wang; Zhencheng Fang; Jie tan; Shufang Wu; Yonghong Xiao; Huaiqiu Zhu

In [6]:
COVID_TERMS = ['covid', 'sars-?n?cov-?2','2019-ncov', 'novel coronavirus', 'sars coronavirus 2']
COVID_SEARCH = f".*({'|'.join(COVID_TERMS)})"
NOVEL_CORONAVIRUS = '.*novel coronavirus'
WUHAN_OUTBREAK = 'wuhan'
    
def tag_covid(data):  
    abstract = data.abstract.fillna('')
    since_covid = (data.published > SARS_COV_2_DATE) | (data.published.isnull())
    covid_term_match = since_covid & abstract.str.match(COVID_SEARCH, case=False)
    wuhan_outbreak = since_covid & abstract.str.match('.*(wuhan|hubei)', case=False)
    covid_match = covid_term_match | wuhan_outbreak
    data['is_covid'] = False
    data.loc[covid_match, 'is_covid'] = True
    return data
    
%time metadata = metadata.pipe(tag_covid)

Wall time: 5.63 s


In [7]:
COLS = ['is_covid', 'covid_related', 'abstract', 'published']
mismatches = metadata[metadata.is_covid != metadata.covid_related]

mismatches =mismatches.loc[(mismatches.published < SARS_COV_2_DATE) &
                          ~(metadata.is_covid), COLS].sort_values('published', ascending=False)
display(len(mismatches))
mismatches

0

,is_covid,covid_related,abstract,published


In [8]:
import re
paper = research_papers[1853]
display(paper.abstract)
display(COVID_SEARCH)
re.match(COVID_SEARCH, paper.abstract, re.IGNORECASE)

'We report co-infection with severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) and influenza A virus in a patient with pneumonia in China. The case highlights possible co-detection of known respiratory viruses. We noted low sensitivity of upper respiratory specimens for SARS-CoV-2, which could further complicate recognition of the full extent of disease.'

'.*(covid|sars-?n?cov-?2|2019-ncov|novel coronavirus|sars coronavirus 2)'

<re.Match object; span=(0, 291), match='We report co-infection with severe acute respirat>

In [9]:
COVID_SEARCH

'.*(covid|sars-?n?cov-?2|2019-ncov|novel coronavirus|sars coronavirus 2)'

In [10]:
research_papers = ResearchPapers.load()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 38.0 seconds


In [11]:
covid_papers = research_papers.covid_related()

In [12]:
len(covid_papers)

2427

In [13]:
covid_papers

,title,abstract,journal,authors,published,when
92,A mathematical model for simulating the transmission of Wuhan novel Coronavirus,"As reported by the World Health Organization, a novel coronavirus (2019-nCoV) was identified as the causative virus of Wuhan pneumonia of unknown etiology by Chinese authorities on 7 January, 2020. In this study, we developed a Bats-Hosts-Reservoir-People transmission network model for simulating the potential transmission from the infection source (probable be bats) to the human infection. Since the Bats-Hosts-Reservoir network was hard to explore clearly and public concerns were focusing on the transmission from a seafood market (reservoir) to people, we simplified the model as Reservoir-People transmission network model. The basic reproduction number (R0) was calculated from the RP model to assess the transmissibility of the 2019-nCoV.",,Tianmu Chen; Jia Rui; Qiupeng Wang; Zeyu Zhao; Jing-An Cui; Ling Yin,2020-01-19,2 months ago
93,"Genomic and protein structure modelling analysis depicts the origin and infectivity of 2019-nCoV, a new coronavirus which caused a pneumonia outbreak in Wuhan, China","Detailed genomic and structure-based analysis of a new coronavirus, namely 2019-nCoV, showed that the new virus is a new type of bat coronavirus and is genetically fairly distant from the human SARS coronavirus. Structure analysis of the spike (S) protein of this new virus showed that its S protein only binds weakly to the ACE2 receptor on human cells whereas the human SARS coronavirus exhibits strongly affinity to the ACE receptor. These findings suggest that the new virus does not readily transmit between humans and should theoretically not able to cause very serious human infection. These data are important to guide design of infection control policy and inform the public on the nature of threat imposed by 2019-nCov when results of direct laboratory tests on this virus are not expected to be available in the near future.",,Ning Dong; Xuemei Yang; Lianwei Ye; Kaichao Chen; Edward Wai-Chi Chan; Mengsu Yang; Sheng Chen,2020-01-21,2 months ago
94,Host and infectivity prediction of Wuhan 2019 novel coronavirus using deep learning algorithm,"The recent outbreak of pneumonia in Wuhan, China caused by the 2019 Novel Coronavirus (2019-nCoV) emphasizes the importance of detecting novel viruses and predicting their risks of infecting people. In this report, we introduced the VHP (Virus Host Prediction) to predict the potential hosts of viruses using deep learning algorithm. Our prediction suggests that 2019-nCoV has close infectivity with other human coronaviruses, especially the severe acute respiratory syndrome coronavirus (SARS-CoV), Bat SARS-like Coronaviruses and the Middle East respiratory syndrome coronavirus (MERS-CoV). Based on our prediction, compared to the Coronaviruses infecting other vertebrates, bat coronaviruses are assigned with more similar infectivity patterns with 2019-nCoVs. Furthermore, by comparing the infectivity patterns of all viruses hosted on vertebrates, we found mink viruses show a closer infectivity pattern to 2019-nCov. These consequences of infectivity pattern analysis illustrate that bat and mink may be two candidate reservoirs of 2019-nCov.These results warn us to beware of 2019-nCoV and guide us to further explore the properties and reservoir of it.One Sentence It is of great value to identify whether a newly discovered virus has the risk of infecting human. Guo et al. proposed a virus host prediction method based on deep learning to detect what kind of host a virus can infect with DNA sequence as input. Applied to the Wuhan 2019 Novel Coronavirus, our prediction demonstrated that several vertebrate-infectious coronaviruses have strong potential to infect human. This method will be helpful in future viral analysis and early prevention and control of viral pathogens.",,Qian Guo; Mo Li; Chunhui Wang; Peihong Wang; Zhencheng Fang; Jie tan; Shufang Wu; Yonghong Xiao; Huaiqiu Zhu

## SARS Labeling

In [18]:
COLS = ['abstract', 'published', 'covid_related', 'virus', 'coronavirus', 'sars']
research_papers = ResearchPapers.load()


Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 45.0 seconds


,abstract,published,covid_related,virus,coronavirus,sars
0,"Next-generation sequencing is increasingly being used to study samples composed of mixtures of organisms, such as in clinical applications where the presence of a pathogen at very low abundance may be highly important. We present an analytical method (SIANN: Strain Identification by Alignment to Near Neighbors) specifically designed to rapidly detect a set of target organisms in mixed samples that achieves a high degree of species- and strain-specificity by aligning short sequence reads to the genomes of near neighbor organisms, as well as that of the target. Empirical benchmarking alongside the current state-of-the-art methods shows an extremely high Positive Predictive Value, even at very low abundances of the target organism in a mixed sample. SIANN is available as an Illumina BaseSpace app, as well as through Signature Science, LLC. SIANN results are presented in a streamlined report designed to be comprehensible to the non-specialist user, providing a powerful tool for rapid species detection in a mixed sample. By focusing on a set of (customizable) target organisms and their near neighbors, SIANN can operate quickly and with low computational requirements while delivering highly accurate results.",2014-01-10,False,False,False,False
1,"An emerging disease is one infectious epidemic caused by a newly transmissible pathogen, which has either appeared for the first time or already existed in human populations, having the capacity to increase rapidly in incidence as well as geographic range. Adapting to human immune system, emerging diseases may trigger large-scale pandemic spreading, such as the transnational spreading of SARS, the global outbreak of A(H1N1), and the recent potential invasion of avian influenza A(H7N9). To study the dynamics mediating the transmission of emerging diseases, spatial epidemiology of networked metapopulation provides a valuable modeling framework, which takes spatially distributed factors into consideration. This review elaborates the latest progresses on the spatial metapopulation dynamics, discusses empirical and theoretical findings that verify the validity of networked metapopulations, and the application in evaluating the effectiveness of disease intervention strategies as well.",2014-06-04,False,False,False,True
2,"Germline variation at immunoglobulin gene (IG) loci is critical for pathogen-mediated immunity, but establishing complete reference sequences in these regions is problematic because of segmental duplications and somatically rearranged source DNA. We sequenced BAC clones from the essentially haploid hydatidiform mole, CHM1, across the light chain IG loci, kappa (IGK) and lambda (IGL), creating single haplotype representations of these regions. The IGL haplotype is 1.25Mb of contiguous sequence with four novel V gene and one novel C gene alleles and an 11.9kbp insertion. The IGK haplotype consists of two 644kbp proximal and 466kbp distal contigs separated by a gap also present in the reference genome sequence. Our effort added an additional 49kbp of unique sequence extending into this gap. The IGK haplotype contains six novel V gene and one novel J gene alleles and a 16.7kbp region with increased sequence identity between the two IGK contigs, exhibiting signatures of interlocus gene conversion. Our data facilitated the first comparison of nucleotide diversity between the light and IG heavy (IGH) chain haplotypes within a single genome, revealing a three to six fold enrichment in the IGH locus, supporting the theory that the heavy chain may be more important in determining antigenic specificity.",2014-07-03,False,False,False,False
3,"Deep sequencing of clinical samples is now an established tool for the detection of infectious pathogens, with direct medical applications. The large amount of data generated provides an opportunity to detect species even at very low levels, provided that computational tools can effectively in

In [27]:
research_papers.since_sars().query('virus')